---
title: "Deep kernel learning"
author: "Andrea Ruglioni"
date: "08 July 2024"
last-modified: "09 July 2024"
description: ""
tags: [machine-learning, gaussian-processes, regression, kernels, multi-output, multi-task, coregionalization, gpytorch, python, tutorial, code, example, deep kernel learning, dkl, deep gaussian processes, dgp, deep learning, neural networks, deep neural networks]
categories: [machine-learning, deep-learning, gaussian-processes]
image: "mgp.png"
toc: true
format: 
  html:
    code-fold: true
nocite: |
  @*
bibliography: refs.bib
---


In this post, we explore Deep Kernel Learning (DKL), a hybrid approach that combines the strengths of deep neural networks (DNNs) with Gaussian Processes (GPs). DKL offers a powerful framework for modeling complex data patterns, enhancing the predictive capabilities and interpretability of standard GPs.

## Gaussian Processes (GPs)

Gaussian Processes are non-parametric models used for regression and classification tasks. A GP is defined by its mean function \( \mu(\mathbf{x}) \) and covariance function (kernel) \( k(\mathbf{x}, \mathbf{x}') \).

$$
f(\mathbf{x}) \sim \mathcal{GP}(\mu(\mathbf{x}), k(\mathbf{x}, \mathbf{x}'))
$$

Given training data \( \mathbf{X} = [\mathbf{x}_1, \ldots, \mathbf{x}_n] \) and targets \( \mathbf{y} \), the predictive distribution for a test point \( \mathbf{x}_* \) is:

$$
p(f_* | \mathbf{X}, \mathbf{y}, \mathbf{x}_*) = \mathcal{N}(\mathbf{k}_*^\top (\mathbf{K} + \sigma^2 \mathbf{I})^{-1} \mathbf{y}, k(\mathbf{x}_*, \mathbf{x}_*) - \mathbf{k}_*^\top (\mathbf{K} + \sigma^2 \mathbf{I})^{-1} \mathbf{k}_*)
$$

where \( \mathbf{K} \) is the kernel matrix for training data, and \( \mathbf{k}_* \) is the covariance vector between \( \mathbf{x}_* \) and training points.

## Deep Kernel Learning (DKL)

DKL integrates DNNs with GPs by using a DNN to learn a representation of the data, which is then used to define the GP kernel. This allows the GP to capture complex patterns in the data that a standard kernel might miss.

### Theory

In DKL, the DNN acts as a feature extractor, transforming input \( \mathbf{x} \) into a feature vector \( \mathbf{z} = \phi(\mathbf{x}) \). The GP kernel is then defined on these features:

$$
k(\mathbf{x}, \mathbf{x}') = k(\phi(\mathbf{x}), \phi(\mathbf{x}'))
$$

The DNN parameters \( \theta \) and the GP hyperparameters are jointly optimized during training.

### Implementation with GPyTorch

Let's implement DKL using GPyTorch.


In [ ]:
%pip install plotly

In [ ]:
import numpy as np
import torch
import gpytorch
import plotly.graph_objects as go
from torch import nn
import matplotlib.pyplot as plt

# Define the McCormick function
def mccormick(x1, x2):
    return np.sin(x1 + x2) + (x1 - x2)**2 - 1.5 * x1 + 2.5 * x2 + 1

# Generate grid data for plotting
x1 = np.linspace(-1.5, 4, 100)
x2 = np.linspace(-3, 4, 100)
x1_test, x2_test = np.meshgrid(x1, x2)
y = mccormick(x1_test, x2_test)

# Generate test data
test_x = torch.tensor(np.vstack((x1_test.ravel(), x2_test.ravel())).T, dtype=torch.float32)
test_y = torch.tensor(y.ravel(), dtype=torch.float32)

# Generate training data
x1 = np.linspace(-1.5, 4, 5)
x2 = np.linspace(-3, 4, 5)
x1_train, x2_train = np.meshgrid(x1, x2)
train_x = np.vstack((x1_train.ravel(), x2_train.ravel())).T
train_y = mccormick(train_x[:, 0], train_x[:, 1])

# Convert training data to tensors
train_x = torch.tensor(train_x, dtype=torch.float32)
train_y = torch.tensor(train_y.ravel(), dtype=torch.float32)

# Define the DNN feature extractor
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 8)
        self.fc3 = nn.Linear(8, 2)
        self.activation = nn.Softplus()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the DKL model
class DKLModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(DKLModel, self).__init__(train_x, train_y, likelihood)
        feature_extractor = FeatureExtractor()
        self.feature_extractor = feature_extractor
        self.num_dims = 2
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        projected_x = self.feature_extractor(x)
        mean_x = self.mean_module(projected_x)
        covar_x = self.covar_module(projected_x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define the GP model
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and models
dkl_likelihood = gpytorch.likelihoods.GaussianLikelihood()
dkl_model = DKLModel(train_x, train_y, dkl_likelihood)
gp_likelihood = gpytorch.likelihoods.GaussianLikelihood()
gp_model = GPModel(train_x, train_y, gp_likelihood)

# Training parameters
training_iterations = 500

# Train DKL model
dkl_model.train()
dkl_likelihood.train()
optimizer = torch.optim.Adam([
    {'params': dkl_model.feature_extractor.parameters()},
    {'params': dkl_model.covar_module.parameters()},
    {'params': dkl_model.mean_module.parameters()},
    {'params': dkl_likelihood.parameters()},
], lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(dkl_likelihood, dkl_model)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = dkl_model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()
    scheduler.step()

# Train GP model
gp_model.train()
gp_likelihood.train()
optimizer = torch.optim.Adam([
    {'params': gp_model.covar_module.parameters()},
    {'params': gp_model.mean_module.parameters()},
    {'params': gp_likelihood.parameters()},
], lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp_likelihood, gp_model)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = gp_model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()
    scheduler.step()

# Evaluate models
dkl_model.eval()
dkl_likelihood.eval()
gp_model.eval()
gp_likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    dkl_pred = dkl_likelihood(dkl_model(test_x))
    gp_pred = gp_likelihood(gp_model(test_x))

dkl_pred = dkl_pred.mean.numpy().reshape(x1_test.shape)
gp_pred = gp_pred.mean.numpy().reshape(x1_test.shape)
true_y = test_y.numpy().reshape(x1_test.shape)

# Create interactive 3D plots
fig = go.Figure()

# DKL predictions surface
fig.add_trace(go.Surface(z=dkl_pred, x=x1_test, y=x2_test, colorscale='Blues', opacity=0.8, name='DKL', showscale=False))

# GP predictions surface
fig.add_trace(go.Surface(z=gp_pred, x=x1_test, y=x2_test, colorscale='Greens', opacity=0.8, name='GP', showscale=False))

# True surface
fig.add_trace(go.Surface(z=true_y, x=x1_test, y=x2_test, colorscale='Reds', opacity=0.5, name='Function', showscale=False))

# Add training points
fig.add_trace(go.Scatter3d(x=train_x[:, 0].numpy(), y=train_x[:, 1].numpy(), z=train_y.numpy(), mode='markers', marker=dict(size=3, color='black', opacity=0.8), name='Train Points'))

fig.update_layout(scene=dict(xaxis_title='X1', yaxis_title='X2', zaxis_title='Y'))

fig.show()

## Comparison with Standard Gaussian Processes

To illustrate the benefits of DKL, we compare its performance with a standard GP on a synthetic dataset. The DKL model captures more complex patterns and provides better uncertainty estimates.


In [ ]:
# get the metrics
from sklearn.metrics import mean_squared_error
import pandas as pd

dkl_mse = mean_squared_error(test_y.numpy(), dkl_pred.ravel())
gp_mse = mean_squared_error(test_y.numpy(), gp_pred.ravel())


df = pd.DataFrame({'Model': ['DKL', 'GP'],
                   'MSE': [dkl_mse, gp_mse])
df

## Concusion

Deep Kernel Learning enhances the flexibility and predictive power of Gaussian Processes by leveraging the feature extraction capabilities of deep neural networks. This hybrid approach captures complex data patterns more effectively than standard GPs, making it a valuable tool for advanced machine learning tasks.